# Link nii files (image-gt pair) and create csv for tracing filenames

In [1]:
import os
import numpy as np
import nibabel as nib
# import matplotlib.pyplot as plt
# %matplotlib inline
import pandas as pd

/home/qeek/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
directory = '/run/user/1000/gvfs/smb-share:server=192.168.200.1,share=mri'
files = os.listdir(os.path.join(directory, 'MRI'))
imgList, gtList = [], []

for f in files:
    if f.endswith('.targets.nii'):
        name = f[:-12]
        print (name)
        for nii in files:
            if name in nii and not nii.endswith('.result.nii') and not nii.endswith('.targets.nii'):
                print (nii)
                imgList.append(nii)
                gtList.append(f)
        print ('---')
    
print ('Total image-gt pair: {}'.format(len(imgList)))
df = pd.DataFrame({'image': imgList, 'target': gtList})
df.to_csv(os.path.join(directory, 'MRI.csv'))

case2015.03.30.13.36.40
case2015.03.30.13.36.40_CHIU_20150330_11527_002_1_21_CYBER_KNIFE.nii
---
case2012.08.29.15.19.35
case2012.08.29.15.19.35_Lu_HungTu_7396_021_NTUH_Gating_3MM_OncoBody_Gated__80_0%_AMP.nii
---
case2016.11.04.13.51.39
---
case2016.11.04.13.54.03
case2016.11.04.13.54.03_CHANG_20161104_1631_005_1_21_CYBER_KNIFE_C+.nii
---
case2009.03.04.13.16.54
---
case2011.04.18.13.08.02
case2011.04.18.13.08.02_YANG_20110418_4663_002_1_21_CYBER_KNIFE.nii
---
case2015.06.16.13.40.29
case2015.06.16.13.40.29_TSAI_20150616_16261_004_1_21_CYBER_KNIFE.nii
---
case2010.05.07.13.43.18
case2010.05.07.13.43.18_GAU_20100507_43586_003_1_21_CYBER_KNIFE.nii
---
case2010.07.26.13.18.50
case2010.07.26.13.18.50_SOUNG_20100726_46207_005_1_21_CYBER_KNIFE_C+.nii
---
case2011.01.18.13.31.24
case2011.01.18.13.31.24_CHEN_20110118_1902_004_1_21_CYBER_KNIFE_C+.nii
---
case2012.08.29.15.17.39
case2012.08.29.15.17.39_Lu_HungTu_7396_013_NTUH_Gating_3MM_OncoBody_Gated__0_0%_AMP.nii
---
case2010.02.08.11.30.30
c

# Slice nii file to original images and target images

In [5]:
from PIL import Image
import os

directory = '/run/user/1000/gvfs/smb-share:server=192.168.200.1,share=mri'
image_directory = os.path.join(directory, 'postMRI', 'image')
if not os.path.exists(image_directory):
    os.makedirs(image_directory)
target_directory = os.path.join(directory, 'postMRI', 'target')
if not os.path.exists(target_directory):
    os.makedirs(target_directory)

# This is for 20180416 data(chest data are in delList)
delList = [1 ,5, 8, 14, 20, 21]
for i, f in enumerate(imgList):
    if i not in delList:
        # Get gt and image file 
        img_data = nib.load(os.path.join(directory, 'MRI', f)).get_data()
        gt_data = nib.load(os.path.join(directory, 'MRI', gtList[i])).get_data()

        # slice every row from top to bottom
        for d in range(gt_data.shape[2]):
            if gt_data[:, :, d].sum() > 0:
                filename = '{:0>4d}_{:0>3d}.png'.format(i, d)
                image = Image.fromarray(img_data[:, :, d]).convert('L')
                image.save(os.path.join(image_directory, filename))
                target = Image.fromarray(gt_data[:, :, d]).convert('L')
                target.save(os.path.join(target_directory, filename))

## Write target images to masks.

In [1]:
from PIL import Image
from dataloader import get_loader, get_data_path
import os
import numpy as np
import scipy.misc as misc


data_loader = get_loader()
data_path = get_data_path()
loader = data_loader(data_path, is_transform=True)

directory = '/run/user/1000/gvfs/smb-share:server=192.168.200.1,share=mri'
split = ['train', 'val']

for s in split:
    mask_directory = os.path.join(directory, 'postMRI', 'mask', s)
    if not os.path.exists(mask_directory):
        os.makedirs(mask_directory)
        
    files = os.listdir(os.path.join(directory, 'postMRI', 'target', s))
    for f in files:
        target = Image.open(os.path.join(directory, 'postMRI', 'target', s, f))
        mask = loader.decode_segmap(np.asarray(target))
        misc.imsave(os.path.join(mask_directory, f), mask)

ModuleNotFoundError: No module named 'torch'

## Apply masks to original images

In [6]:
import os
from PIL import Image

directory = '/run/user/1000/gvfs/smb-share:server=192.168.200.1,share=mri'
split = ['train', 'val']

for s in split:
    map_directory = os.path.join(directory, 'postMRI', 'map', s)
    if not os.path.exists(map_directory):
        os.makedirs(map_directory)
    
    files = os.listdir(os.path.join(directory, 'postMRI', 'image', s))
    for f in files:
        bg = Image.open(os.path.join(directory, 'postMRI', 'image', s, f)).convert('RGB')
        fg = Image.open(os.path.join(directory, 'postMRI', 'mask', s, f))
        Image.blend(bg, fg, alpha=0.5).save(os.path.join(map_directory, f))

# Merge masks and origin images side by side

In [8]:
import matplotlib.pyplot as plt
from PIL import Image
import os

directory = '/run/user/1000/gvfs/smb-share:server=192.168.200.1,share=mri'
split = ['train', 'val']

for s in split:
    filenames = os.listdir(os.path.join(directory, 'postMRI', 'image', s))
    for f in filenames:
        img1 = Image.open(os.path.join(directory, 'postMRI', 'image', s, f))
        img2 = Image.open(os.path.join(directory, 'postMRI', 'map', s, f))
        new = Image.new('RGB', (img1.size[0]*2+10, img1.size[1]), (255, 255, 255))
        new.paste(img1, (0, 0))
        new.paste(img2, (img1.size[0]+10, 0))
        new.save(os.path.join(directory, 'postMRI', 'merge-image-map', s, f))

In [1]:
import matplotlib.pyplot as plt
from PIL import Image
import os

directory = os.getcwd()

filenames = os.listdir(os.path.join(directory, 'prediction', 'model_006_prediction', 'gt'))
for f in filenames:
    img1 = Image.open(os.path.join(directory, 'prediction', 'model_006_prediction', 'gt', f))
    img2 = Image.open(os.path.join(directory, 'prediction', 'model_006_prediction', 'pred', f))
    new = Image.new('RGB', (img1.size[0]*2+10, img1.size[1]), (255, 255, 255))
    new.paste(img1, (0, 0))
    new.paste(img2, (img1.size[0]+10, 0))
    new.save(os.path.join(directory, 'prediction', 'model_006_prediction', 'merge', f))

## Apply sliding windows through images for retrieve proper train/val data

In [6]:
import numpy as np

def sliding_window(image, step, windowSize):
    image = np.asarray(image)
    for y in range(0, image.shape[0], step):
        for x in range(0, image.shape[1], step):
            yield (x, y, image[y:y+windowSize[1], x:x+windowSize[0]])

In [3]:
import os
from PIL import Image
import matplotlib.pyplot as plt

directory = '/run/user/1000/gvfs/smb-share:server=192.168.200.1,share=mri'
split = ['train', 'val']

step = 20
windowSize = (64, 64)
for s in split:
    win_img_directory = os.path.join(directory, 'postMRI', 'window_image', s)
    if not os.path.exists(win_img_directory):
        os.makedirs(win_img_directory)
    
    win_gt_directory = os.path.join(directory, 'postMRI', 'window_target', s)
    if not os.path.exists(win_gt_directory):
        os.makedirs(win_gt_directory)    
 
    win_mask_directory = os.path.join(directory, 'postMRI', 'window_mask', s)
    if not os.path.exists(win_mask_directory):
        os.makedirs(win_mask_directory)
        
    files = os.listdir(os.path.join(directory, 'postMRI', 'target', s))
    for f in files:
        windowCount = 0
        target = Image.open(os.path.join(directory, 'postMRI', 'target', s, f))
        image = Image.open(os.path.join(directory, 'postMRI', 'image', s, f))
        mask = Image.open(os.path.join(directory, 'postMRI', 'mask', s, f))

        for (x, y, gt_patch) in sliding_window(target, step=step, windowSize=windowSize):
            if gt_patch.sum() > 200:
                filename = '{}_{:0>5d}.png'.format(f[:-4], windowCount)
                gt_patch = Image.fromarray(gt_patch)
                gt_patch.save(os.path.join(win_gt_directory, filename))
                
                image = np.asarray(image)
                img_patch = image[y:y+windowSize[1], x:x+windowSize[0]]
                img_patch = Image.fromarray(img_patch)
                img_patch.save(os.path.join(win_img_directory, filename))
      
                mask = np.asarray(mask)
                mask_patch = mask[y:y+windowSize[1], x:x+windowSize[0]]
                mask_patch = Image.fromarray(mask_patch)
                mask_patch.save(os.path.join(win_mask_directory, filename))
                
                windowCount += 1

# Merge gt and prediction masks side by side

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os

filenames = os.listdir(os.path.join(os.getcwd(), 'prediction', 'pred'))
for f in filenames:
    new = Image.new('RGB', (64*2+10, 64), (255, 255, 255))
    img1 = Image.open(os.path.join(os.getcwd(), 'prediction', 'gt', f))
    img2 = Image.open(os.path.join(os.getcwd(), 'prediction', 'pred', f))
    new.paste(img1, (0, 0))
    new.paste(img2, (64+10, 0))
    new.save(os.path.join(os.getcwd(), 'prediction', 'merge', f))

# Apply window masks to original images

In [21]:
import matplotlib.pyplot as plt
from PIL import Image
import os

step = 20
windowSize = (64, 64)

filenames = os.listdir(os.path.join(os.getcwd(), 'prediction', 'image', 'val'))
for f in filenames:
    target = Image.open(os.path.join(os.getcwd(), 'prediction', 'target', 'val', f))
    bg = Image.open(os.path.join(os.getcwd(), 'prediction', 'image', 'val', f)).convert('RGB')
    fg = np.zeros((target.size[0], target.size[1], 3))
    windowCount = 0
    for (x, y, gt_patch) in sliding_window(target, step=step, windowSize=windowSize):
        if gt_patch.sum() > 200:
            window_filename = '{}_{:0>5d}.png'.format(f[:-4], windowCount)
            fg[y:y+windowSize[1], x:x+windowSize[0]] += np.asarray(Image.open(os.path.join(os.getcwd(), 'prediction', 'pred', window_filename)))
            windowCount += 1

    fg = Image.fromarray(fg.astype(dtype=np.uint8))
    Image.blend(bg, fg, alpha=0.5).save(os.path.join(os.getcwd(), 'prediction', 'pred-map', f))

# Merge gt and prediction masks side by side

In [22]:
import matplotlib.pyplot as plt
from PIL import Image
import os

filenames = os.listdir(os.path.join(os.getcwd(), 'prediction', 'pred-map'))
for f in filenames:
    img1 = Image.open(os.path.join(os.getcwd(), 'prediction', 'map', 'val', f))
    img2 = Image.open(os.path.join(os.getcwd(), 'prediction', 'pred-map', f))
    new = Image.new('RGB', (img1.size[0]*2+10, img1.size[1]), (255, 255, 255))

    new.paste(img1, (0, 0))
    new.paste(img2, (img1.size[0]+10, 0))
    new.save(os.path.join(os.getcwd(), 'prediction', 'merge-map', f))